<div align="center">

# CMSC 320 Final Project: How Much Energy Does the United States Consume?

##### A Tutorial and Analysis by Jay Desmarais
***

</div>

<div align="center">

### Introduction

</div>

blah blah blah

<div align="center">

### Required Tools and Libraries

</div>

The following tools and libraries are required for different aspects of this tutorial:

1. Requests 
2. pandas
3. numpys
he


To import these packages, the following code can be used: 

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'requests'